In [1]:
import pickle
import math, time
import numpy as np
from param import Parameters
from components import *
from initiate import add_vehicles_from_data, add_requests_from_data#,add_requests_from_data2
from opt_ridesharing import *

In [2]:
def init_params():
    '''
    init the params.
    :return:
    '''
    Parameters.Requests = []
    Parameters.RequestIndex = {}
    Parameters.Vehicles = []
    Parameters.RequestId = 0
    Parameters.VehicleId = 0
    Parameters.Trips = {}
    Parameters.RequestDone = []
    Parameters.computation_time = 0
    Parameters.IdleVehicles = []
    Parameters.unassigned_requests = []
    Parameters.VehiclePosition = {}
    Parameters.fea_set = {}
    Parameters.fea_set_last_iteration = {}
    Parameters.current_request_id = 0
    Parameters.uber_percent = 0
    Parameters.uber_vehicles = 0
    Parameters.uber_request_percent = 0

In [3]:
init_params()
Demand = pickle.load(open("./expdata1/demand20160109","rb"))
print(len(Demand))

397847


In [4]:
# params
Parameters.uberpool_vehicles = 2500
Parameters.MaxWaiting = 300
Parameters.MaxDelay = 600
Parameters.TimeWindow = 30
Parameters.num_time_windows = int(3600/Parameters.TimeWindow)

add_vehicles_from_data(3600*0+30, Demand)

In [5]:
def simulation_ridesharing(time_start, demands):
    time_now = time_start + Parameters.TimeWindow
    Parameters.TimeEnd = time_now + Parameters.TimeWindow * Parameters.num_time_windows

    rounds = 1
    while (time_now < Parameters.TimeEnd + math.inf):
        print("\nrounds:", rounds, "time_now:", time_now)
        if time_now < Parameters.TimeEnd:
            demands = add_requests_from_data((time_now - Parameters.TimeWindow, time_now), demands)
            #demands = add_requests_from_data2((time_now - Parameters.TimeWindow, time_now), demands)
#         print("orders num in this timestamp:", len(Parameters.Requests))

        if len(Parameters.Requests) == 0:
            finished = True
            for r in Parameters.RequestDone:
                if r.ignored:
                    continue
                else:
                    if r.DoTime == None:
                        finished = False
                        break
            if finished:
                break

        rr = combine_rr(time_now)

        compute_RV(time_now)
        Parameters.fea_set_last_iteration = Parameters.fea_set
        Parameters.fea_set = {}

        compute_RTV(rr)
        opt_assignment(time_now)

        time_now = time_now + Parameters.TimeWindow
        rounds += 1
        Parameters.current_request_id = Parameters.RequestId


In [6]:
st = time.time()
simulation_ridesharing(3600*0, Demand)
end = time.time()


rounds: 1 time_now: 30
Academic license - for non-commercial use only
reblance_requests 0
idling_vehicles 2389

rounds: 2 time_now: 60
reblance_requests 1
idling_vehicles 2267

rounds: 3 time_now: 90
reblance_requests 0
idling_vehicles 2161

rounds: 4 time_now: 120
reblance_requests 0
idling_vehicles 2053

rounds: 5 time_now: 150
reblance_requests 0
idling_vehicles 1955

rounds: 6 time_now: 180
reblance_requests 0
idling_vehicles 1841

rounds: 7 time_now: 210
reblance_requests 1
idling_vehicles 1750

rounds: 8 time_now: 240
reblance_requests 1
idling_vehicles 1663

rounds: 9 time_now: 270
reblance_requests 0
idling_vehicles 1578

rounds: 10 time_now: 300
reblance_requests 0
idling_vehicles 1493

rounds: 11 time_now: 330
reblance_requests 0
idling_vehicles 1400

rounds: 12 time_now: 360
reblance_requests 1
idling_vehicles 1324

rounds: 13 time_now: 390
reblance_requests 0
idling_vehicles 1255

rounds: 14 time_now: 420
reblance_requests 0
idling_vehicles 1191

rounds: 15 time_now: 450
r

KeyError: 3536

In [0]:
# save results
n_v = Parameters.uberpool_vehicles
wait = Parameters.MaxWaiting
delay = Parameters.MaxDelay
window = Parameters.TimeWindow
n_window = Parameters.num_time_windows

result = Parameters.RequestDone
d1 = open(f"./trainingdata2/r_0109_{n_v}_{wait}_{delay}_{window}_{n_window}","wb")
pickle.dump(result,d1)
d1.close()

v_result = Parameters.Vehicles
v1 = open(f"./trainingdata2/v_0109_{n_v}_{wait}_{delay}_{window}_{n_window}","wb")
pickle.dump(v_result,v1)
v1.close()

print(f"time cost: {end-st} s")

- Parameters.MaxWaiting = 300
- Parameters.MaxDelay = 600
- Parameters.num_time_windows = 120

## exp1

### uberpool_vehicles

#### Mean service rate

| uberpool_vehicles | request done | ignore | pick up |
| --- | --- | --- | --- |
| 300 | 20492 | 18427 | 2065 |
| 600 | 20492 | 16191 | 6366 |

#### Mean distance travelled
| uberpool_vehicles | travel time | reblance time | idle time |
| --- | --- | --- | --- |
| 300 | 2125735 | 73592 | 797137 |
| 600 | 4198113 | 147995 | 1590110 |

### Percentage of shared rides
| uberpool_vehicles | share_dis - dis / dis |
| --- | --- |
| 300 | 0.139 |
| 600 | 0.1347 |

#### Mean dropoff delay
| uberpool_vehicles | average do delay |
| --- | --- |
| 300 | 258.798 |
| 600 | 256.147 |

#### Mean pickup delay
| uberpool_vehicles | average pu delay |
| --- | --- |
| 300 | 132.838 |
| 600 | 141.191 |


In [ ]:
# service rate
count = 0
for r in Parameters.RequestDone:
    if r.ignored is True:
        count += 1
print("all num:", len(Parameters.RequestDone))
print("ignored:", count)
print("picked num:", Parameters.PU)

# share rate
count1 = 0
count5 = 0
for r in Parameters.RequestDone:
    if r.IsUber is False and r.ignored is False:
        count1 += 1
    if r.shared is True:
        count5 += 1
print("choose share:", count1, "real share:", count5)

# travel time
count6 = 0
count7 = 0
count8 = 0
for v in Parameters.Vehicles:
    count6 += v.TravelTime    
    count7 += v.RbTotalTime
    count8 += v.IdleTotalTime

print("average travel time:", count6/len(Parameters.Vehicles))
print("average reblance time:", count7/len(Parameters.Vehicles))
print("average idle time:", count8/len(Parameters.Vehicles))

# 共享路程比
r_sum = 0
r_num = 0
for r in Parameters.RequestDone:
    if r.ignored is False and r.shared is True:
        sharedis = r.share_distance[1]+r.share_distance[2]
        
        dis = Parameters.DistanceDict[r.PuHub][r.DoHub]
        ratio = (sharedis - dis)/dis
        r_sum += ratio
        r_num += 1

print("share dist ratio:", r_sum/r_num)

# Dropoff delay
sum = 0
num = 0
for r in Parameters.RequestDone:
    if r.ignored is False:
        sum += (r.DoTime-r.EarliestDoTime)
        num += 1
print("average dropoff delay:", sum/num)

# pick up delay
sum = 0
num = 0
for r in Parameters.RequestDone:
    if r.ignored is False:
        sum += (r.PuTime-r.RequestTime)
        num += 1
print("average pickup delay:", sum/num)